In [1]:
import pandas as pd

In [2]:
datos = pd.read_csv('../Archivos/crimenes-GAM-todos.csv')
datos.head()

,fecha,crimen,categoria,colonia,long,lat
0,2016-01-05 17:00:00,LESIONES CULPOSAS,DELITO DE BAJO IMPACTO,GRANJAS MODERNAS,-99.103527,19.481076
1,2016-09-23 12:00:00,FRAUDE,DELITO DE BAJO IMPACTO,LA PRADERA,-99.071243,19.475715
2,2017-10-31 21:30:00,DAÑO EN PROPIEDAD AJENA INTENCIONAL A AUTOMOVIL,DELITO DE BAJO IMPACTO,NUEVA INDUSTRIAL VALLEJO,-99.147318,19.500086
3,2018-05-21 09:00:00,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,GABRIEL HERNÁNDEZ,-99.099081,19.498720
4,2016-01-07 08:55:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,CONSTITUCIÓN DE LA REPÚBLICA,-99.105545,19.482936


In [3]:
datos.shape

(80097, 6)

In [4]:
datos.dtypes

fecha         object
crimen        object
categoria     object
colonia       object
long         float64
lat          float64
dtype: object

In [5]:
crimenes = datos.convert_dtypes()
crimenes.dtypes

fecha         string
crimen        string
categoria     string
colonia       string
long         float64
lat          float64
dtype: object

In [6]:
crimenes['categoria'].unique()

<StringArray>
[                                 'DELITO DE BAJO IMPACTO',
    'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA',
                                      'HECHO NO DELICTIVO',
                   'ROBO A REPARTIDOR CON Y SIN VIOLENCIA',
                    'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA',
           'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO',
 'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA',
                            'ROBO A NEGOCIO CON VIOLENCIA',
                    'ROBO A CASA HABITACIÓN CON VIOLENCIA',
                                        'HOMICIDIO DOLOSO',
 'ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA',
   'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA',
                                               'VIOLACIÓN',
           'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA',
                'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA',
                                               'SECUESTRO']
Length: 16, dtype: string

In [7]:
crimenes.groupby(['categoria']).size().to_frame().reset_index().rename(columns={
    0:'cuenta_delito'
}).sort_values(by='cuenta_delito',ascending=True)

,categoria,cuenta_delito
14,SECUESTRO,11
12,ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA,62
8,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,74
5,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,182
15,VIOLACIÓN,218
4,ROBO A CASA HABITACIÓN CON VIOLENCIA,241
9,ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VI...,241
7,ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN ...,419
2,HOMICIDIO DOLOSO,739
10,ROBO A REPARTIDOR CON Y SIN VIOLENCIA,1067


In [8]:
def tabla_frecuencias_categoria(categoria, ascendente):
    return crimenes[crimenes['categoria']==categoria].groupby(['crimen']).size().to_frame().reset_index().rename(columns={
    0:'Total'
}).sort_values(by='Total',ascending=~ascendente)

In [9]:
pd.isnull(crimenes['fecha']).values.ravel().sum()

14

# Secuestro

In [10]:
tabla_frecuencias_categoria('SECUESTRO', True)

,crimen,Total
0,PLAGIO O SECUESTRO,11


# Robo a transportista con y sin violencia

In [11]:
tabla_frecuencias_categoria('ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA', True)

,crimen,Total
1,ROBO A TRANSPORTISTA Y VEHICULO PESADO SIN VIO...,6
0,ROBO A TRANSPORTISTA Y VEHICULO PESADO CON VIO...,56


 # Robo a pasajero a bordo de taxi con violencia

In [12]:
tabla_frecuencias_categoria('ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA', True)

,crimen,Total
0,ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA,74


# Robo a cuentahabiente saliendo del cajero con violencia

In [13]:
tabla_frecuencias_categoria('ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA', True)

,crimen,Total
1,ROBO A TRANSEUNTE SALIENDO DEL CAJERO CON VIOL...,20
0,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,162


# Violación

In [14]:
tabla_frecuencias_categoria('VIOLACIÓN', True)

,crimen,Total
2,VIOLACION TUMULTUARIA,5
1,VIOLACION EQUIPARADA,24
0,VIOLACION,189


In [15]:
crimenes = crimenes[crimenes['categoria']!='VIOLACIÓN']
crimenes.shape

(79879, 6)

# Robo a casa habitación con violencia

In [16]:
tabla_frecuencias_categoria('ROBO A CASA HABITACIÓN CON VIOLENCIA', True)

,crimen,Total
1,ROBO A CASA HABITACION Y VEHICULO CON VIOLENCIA,36
0,ROBO A CASA HABITACION CON VIOLENCIA,205


#  Robo a pasajero a bordo del metro con y sin violencia

In [17]:
tabla_frecuencias_categoria('ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA', True)

,crimen,Total
0,ROBO A PASAJERO A BORDO DE METRO CON VIOLENCIA,39
1,ROBO A PASAJERO A BORDO DE METRO SIN VIOLENCIA,202


# Robo a pasajero a bordo de microbus con y sin violencia

In [18]:
tabla_frecuencias_categoria('ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA', True)

,crimen,Total
1,ROBO A PASAJERO A BORDO DE PESERO COLECTIVO SI...,32
0,ROBO A PASAJERO A BORDO DE PESERO COLECTIVO CO...,387


# Homicidio doloso

In [19]:
tabla_frecuencias_categoria('HOMICIDIO DOLOSO', True)

,crimen,Total
1,HOMICIDIO POR AHORCAMIENTO,1
0,FEMINICIDIO,18
4,HOMICIDIO POR GOLPES,33
5,HOMICIDIOS INTENCIONALES (OTROS),47
2,HOMICIDIO POR ARMA BLANCA,69
3,HOMICIDIO POR ARMA DE FUEGO,571


# Robo a repartidor con y sin violencia

In [20]:
tabla_frecuencias_categoria('ROBO A REPARTIDOR CON Y SIN VIOLENCIA', True)

,crimen,Total
3,ROBO A REPARTIDOR Y VEHICULO SIN VIOLENCIA,12
2,ROBO A REPARTIDOR Y VEHICULO CON VIOLENCIA,62
1,ROBO A REPARTIDOR SIN VIOLENCIA,133
0,ROBO A REPARTIDOR CON VIOLENCIA,860


# Lesiones dolosas por disparo de arma de fuego

In [21]:
tabla_frecuencias_categoria('LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO', True)

,crimen,Total
0,LESIONES INTENCIONALES POR ARMA DE FUEGO,1111


# Robo a negocio con violencia

In [22]:
tabla_frecuencias_categoria('ROBO A NEGOCIO CON VIOLENCIA', True)

,crimen,Total
4,ROBO A SUCURSAL BANCARIA (SUPERMERCADO) CON VI...,1
5,ROBO DE VEHICULO Y NOMINA CON VIOLENCIA,1
6,ROBO DURANTE TRASLADO DE VALORES (NOMINA) CON ...,1
1,ROBO A NEGOCIO Y VEHICULO CON VIOLENCIA,4
2,ROBO A OFICINA PÚBLICA CON VIOLENCIA,4
3,ROBO A SUCURSAL BANCARIA (ASALTO BANCARIO) CON...,4
0,ROBO A NEGOCIO CON VIOLENCIA,1354


# Robo a transeunte en  vía pública con y sin violencia

In [23]:
tabla_frecuencias_categoria('ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA', True)

,crimen,Total
0,ROBO A TRANSEUNTE EN VIA PUBLICA (NOMINA) CON ...,4
2,ROBO A TRANSEUNTE EN VIA PUBLICA SIN VIOLENCIA,410
1,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,3110


# Hecho no delictivo

In [24]:
tabla_frecuencias_categoria('HECHO NO DELICTIVO', True)

,crimen,Total
15,PERDIDA DE LA VIDA POR DERRUMBE,1
8,DENUNCIA DE HECHOS POR ROBO DE CELULAR,1
0,DDH ANONIMAS,3
4,DDH REDES,3
12,PERDIDA DE LA VIDA POR ASFIXIA,3
9,PERDIDA DE LA VIDA ASFIXIA POR ALIMENTOS/ LIQU...,3
1,DDH CERESO,4
2,DDH FRVT,5
10,PERDIDA DE LA VIDA POR ACCIDENTE LABORAL,5
23,PERDIDA DE LA VIDA POR SUICIDIO EN EL METRO,6


# Robo de vehículo con y sin violencia

In [25]:
tabla_frecuencias_categoria('ROBO DE VEHÍCULO CON Y SIN VIOLENCIA', True)

,crimen,Total
0,ROBO DE MAQUINARIA CON VIOLENCIA,2
5,ROBO DE VEHICULO DE SERVICIO OFICIAL CON VIOLE...,5
6,ROBO DE VEHICULO DE SERVICIO OFICIAL SIN VIOLE...,20
3,ROBO DE VEHICULO DE SERVICIO DE TRANSPORTE CON...,23
4,ROBO DE VEHICULO DE SERVICIO DE TRANSPORTE SIN...,31
10,ROBO DE VEHICULO DE SERVICIO PÚBLICO SIN VIOLE...,334
2,ROBO DE MOTOCICLETA SIN VIOLENCIA,376
1,ROBO DE MOTOCICLETA CON VIOLENCIA,411
9,ROBO DE VEHICULO DE SERVICIO PÚBLICO CON VIOLE...,682
7,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,1460


# Delito de bajo impacto

In [26]:
tabla_frecuencias_categoria('DELITO DE BAJO IMPACTO',True)[40:60]

,crimen,Total
101,ROBO A NEGOCIO Y VEHICULO SIN VIOLENCIA,6
118,ROBO A SUCURSAL BANCARIA (SUPERMERCADO) SIN VI...,6
154,TRATA DE PERSONAS,6
150,TENTATIVA DE VIOLACION,6
20,CORRUPCION DE MENORES E INCAPACES,6
84,PECULADO,7
114,ROBO A PASAJERO EN RTP CON VIOLENCIA,7
11,BIGAMIA,7
43,ESTUPRO,7
83,OTROS CULPOSOS,8
